# Lab 3: Streaming - Respostas em Tempo Real

<img src="./assets/LC_streaming.png" width="400">

## 🎯 O que você vai aprender neste Lab

**Streaming** reduz a latência percebida pelo usuário, entregando dados assim que são gerados pelo modelo, em vez de esperar a resposta completa.

### Por que Streaming é importante?

1. **Melhor UX**: Usuários veem progresso imediato (como no ChatGPT)
2. **Menor latência percebida**: Primeira palavra aparece rapidamente
3. **Feedback visual**: Usuário sabe que o sistema está trabalhando
4. **Aplicações interativas**: Essencial para chatbots e assistentes

### Modos de Streaming no LangChain:

| Modo | Descrição | Uso |
|------|-----------|-----|
| `values` | Retorna após cada passo do agente | Debug, visualizar ciclo ReAct |
| `messages` | Token por token | Chatbots, UX responsiva |
| `custom` | Dados customizados de tools | Progresso de operações longas |

## ⚙️ Setup - Configuração do Ambiente

Instalação das bibliotecas necessárias.

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langgraph databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
from langchain.agents import create_agent

In [ ]:
import mlflow
mlflow.langchain.autolog()

In [ ]:
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian",
)

## Criando o Agente de Teste

Vamos criar um agente simples para demonstrar os diferentes modos de streaming.

## 🚫 Sem Streaming (`invoke`)

Primeiro, vamos ver como é **sem streaming**. O método `invoke()` espera a resposta completa antes de retornar.

**Problema**: O usuário não vê nada até que toda a resposta esteja pronta. Para respostas longas, isso pode parecer lento.

In [0]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

## 📊 Modo `values` - Streaming por Passos

Este é o modo que usamos nos exemplos anteriores. Retorna dados **após cada passo** do ciclo ReAct:
- Após o modelo raciocinar
- Após uma tool ser chamada
- Após a resposta final

**Ideal para**: Debug, visualizar o fluxo de trabalho do agente, entender o raciocínio.

In [0]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## ⚡ Modo `messages` - Token por Token

Este é o modo mais responsivo! Retorna cada token assim que é gerado pelo LLM.

**É exatamente assim que ChatGPT e Claude funcionam** - você vê as palavras aparecendo uma a uma.

**Ideal para**: Chatbots, interfaces conversacionais, qualquer aplicação onde UX responsiva é importante.

In [0]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

## 🔧 Modo `custom` - Streaming de Tools

Tools também podem fazer streaming! Isso é útil quando uma ferramenta executa uma operação demorada e você quer mostrar progresso ao usuário.

### Como funciona:
1. Use `get_stream_writer()` dentro da tool
2. Chame `writer("mensagem")` para enviar dados
3. Use `stream_mode=["custom"]` para receber esses dados

**Ideal para**: 
- Operações longas (downloads, processamento)
- Feedback de progresso
- Log de etapas intermediárias

In [ ]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent_weather = create_agent(
    model=llm,
    tools=[get_weather],
)

for chunk in agent_weather.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

### Usando apenas o modo `custom`

Você pode filtrar para receber apenas os dados customizados da tool:

## 🎮 Sua Vez - Experimente!

Combine diferentes modos de streaming e filtre os resultados como preferir.

### Filtrando apenas dados custom:

Quando você usa múltiplos modos, cada chunk vem como uma tupla `(modo, dados)`. Você pode filtrar:

## Try different modes on your own!
Modify the stream mode and the select to produce different results.

## 📚 Resumo e Próximos Passos

### O que aprendemos:

| Método | Modo | Latência | Uso |
|--------|------|----------|-----|
| `invoke()` | Nenhum | Alta | Operações batch |
| `stream(mode="values")` | Por passo | Média | Debug, visualização |
| `stream(mode="messages")` | Por token | Baixa | Chatbots, UX |
| `stream(mode="custom")` | Customizado | Variável | Progresso de tools |

### Dicas:
- Use `messages` para a melhor experiência do usuário
- Use `values` para debugging e entender o agente
- Use `custom` para operações longas em tools
- Combine modos quando precisar de diferentes informações

### Próximo Lab:
No **Lab 4 - Tools**, você aprenderá a criar ferramentas mais sofisticadas com descrições detalhadas e comportamentos customizados!

In [ ]:
for chunk in agent_weather.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])